In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
os.__file__

'C:\\ProgramData\\Anaconda3\\envs\\pythonProject1\\lib\\os.py'

In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
# 随机森林优点，不容易出现过拟合，特点:可以不用处理缺失值
import os
os.chdir(r'C:\Users\fzh00\Desktop\文件\excel\阿里天池\贷款违约预测')

In [3]:
train = pd.read_csv('train.csv')
train.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000,5,19.52,917.97,E,E2,320.0,2 years,2,...,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0
1,1,18000,5,18.49,461.90,D,D2,219843.0,5 years,0,...,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN
2,2,12000,5,16.99,298.17,D,D3,31698.0,8 years,0,...,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0
3,3,11000,3,7.26,340.96,A,A4,46854.0,10+ years,1,...,16.0,4.0,7.0,21.0,6.0,9.0,0.0,0.0,0.0,1.0
4,4,3000,3,12.99,101.07,C,C2,54.0,NaN,1,...,4.0,9.0,10.0,15.0,7.0,12.0,0.0,0.0,0.0,4.0


In [4]:
data = train.copy()
data.drop(['id','employmentTitle','ficoRangeLow','ficoRangeHigh','title','policyCode'],axis=1,inplace=True)
for i in data.columns:
    if data[i].dtype =='object':
        data.drop(i,inplace = True,axis=1)
data.shape
data.dropna(axis=0,inplace= True)
data.shape

(800000, 36)

(729535, 36)

In [8]:
data.isnull().sum()

loanAmnt              0
term                  0
interestRate          0
installment           0
homeOwnership         0
annualIncome          0
verificationStatus    0
isDefault             0
purpose               0
postCode              0
regionCode            0
dti                   0
delinquency_2years    0
openAcc               0
pubRec                0
pubRecBankruptcies    0
revolBal              0
revolUtil             0
totalAcc              0
initialListStatus     0
applicationType       0
n0                    0
n1                    0
n2                    0
n3                    0
n4                    0
n5                    0
n6                    0
n7                    0
n8                    0
n9                    0
n10                   0
n11                   0
n12                   0
n13                   0
n14                   0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
y = data.pop('isDefault')
xtrain,xtest,ytrain,ytest = train_test_split(data,y,test_size=0.2,stratify=y)

In [6]:
from sklearn.linear_model import LogisticRegression
# 模型评估（召回率），选择0.5为阈值
from sklearn.metrics import classification_report # 用于分类报告
# 模型评估
from sklearn import metrics
from sklearn.model_selection import train_test_split,cross_val_score	#划分数据 交叉验证
score = cross_val_score(LogisticRegression(),xtrain,ytrain,cv=5)
type(score)

numpy.ndarray

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import time

clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = AdaBoostClassifier()
clf4 = GradientBoostingClassifier()
clf5 = XGBClassifier(eval_metric='mlogloss')#这里不添加里面的参数会产生警告，原因版本问题objective ‘binary:logistic’ was changed from ‘error’ to ‘logloss’.
clf6 = LGBMClassifier()

for clf, label in zip([clf1, clf2, clf3, clf4, clf5, clf6], [
        'Logistic Regression', 'Random Forest', 'AdaBoost', 'GBDT', 'XGBoost',
        'LightGBM'
]):
    start = time.time()
    scores = cross_val_score(clf, xtrain,ytrain, scoring='accuracy', cv=5)
    end = time.time()
    running_time = end - start
    print("Accuracy: %0.8f (+/- %0.2f),耗时%0.2f秒。模型名称[%s]" %
          (scores.mean(), scores.std(), running_time, label))

Accuracy: 0.80059558 (+/- 0.00),耗时28.18秒。模型名称[Logistic Regression]
Accuracy: 0.80298409 (+/- 0.00),耗时883.33秒。模型名称[Random Forest]
Accuracy: 0.80315543 (+/- 0.00),耗时257.72秒。模型名称[AdaBoost]
Accuracy: 0.80372600 (+/- 0.00),耗时947.59秒。模型名称[GBDT]
Accuracy: 0.80416293 (+/- 0.00),耗时126.28秒。模型名称[XGBoost]
Accuracy: 0.80465639 (+/- 0.00),耗时16.38秒。模型名称[LightGBM]


In [10]:
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [11]:
from sklearn.linear_model import LogisticRegression
# 模型评估（召回率），选择0.5为阈值
from sklearn.metrics import classification_report # 用于分类报告
# 模型评估
from sklearn import metrics

LR = LogisticRegression()
LR.fit(xtrain,ytrain)
LR.score(xtest,ytest)

fpr, tpr, th = metrics.roc_curve(ytest, LR.predict(xtest))
metrics.auc(fpr, tpr)

#输出分类报告，Y的真实值，Y的预测标签
print(classification_report(ytest, LR.predict(xtest)))

LogisticRegression()

0.8005030601684635

0.5001817029480845

              precision    recall  f1-score   support

           0       0.80      1.00      0.89    116832
           1       0.30      0.00      0.00     29075

    accuracy                           0.80    145907
   macro avg       0.55      0.50      0.45    145907
weighted avg       0.70      0.80      0.71    145907



In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(xtrain,ytrain)

knn.score(xtest,ytest)
fpr_knn, tpr_knn, th_knn = metrics.roc_curve(ytest, LR.predict(xtest))
metrics.auc(fpr_knn, tpr_knn)
#输出分类报告，Y的真实值，Y的预测标签
print(classification_report(ytest, knn.predict(xtest)))


KNeighborsClassifier()

0.7671324885029505

0.5001817029480845

              precision    recall  f1-score   support

           0       0.80      0.94      0.87    116832
           1       0.25      0.09      0.13     29075

    accuracy                           0.77    145907
   macro avg       0.53      0.51      0.50    145907
weighted avg       0.69      0.77      0.72    145907



In [13]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(xtrain,ytrain)

svc.score(xtest,ytest)
fpr_svc, tpr_svc, th_svc = metrics.roc_curve(ytest, svc.predict(xtest))
metrics.auc(fpr_svc, tpr_svc)
#输出分类报告，Y的真实值，Y的预测标签
print(classification_report(ytest, svc.predict(xtest)))

LinearSVC()

0.8007360853146183

0.5000171969045571

              precision    recall  f1-score   support

           0       0.80      1.00      0.89    116832
           1       1.00      0.00      0.00     29075

    accuracy                           0.80    145907
   macro avg       0.90      0.50      0.44    145907
weighted avg       0.84      0.80      0.71    145907



In [14]:
from sklearn.naive_bayes import GaussianNB
bay = GaussianNB()
bay.fit(xtrain,ytrain)

bay.score(xtest,ytest)
fpr_bay, tpr_bay, th_bay = metrics.roc_curve(ytest, bay.predict(xtest))
metrics.auc(fpr_bay, tpr_bay)
#输出分类报告，Y的真实值，Y的预测标签
print(classification_report(ytest, bay.predict(xtest)))

GaussianNB()

0.7786398185145332

0.5492816468261678

              precision    recall  f1-score   support

           0       0.82      0.93      0.87    116832
           1       0.38      0.17      0.23     29075

    accuracy                           0.78    145907
   macro avg       0.60      0.55      0.55    145907
weighted avg       0.73      0.78      0.74    145907



In [15]:
from sklearn import tree
DT = tree.DecisionTreeClassifier()
DT.fit(xtrain,ytrain)

DT.score(xtest,ytest)
fpr_DT, tpr_DT, th_DT = metrics.roc_curve(ytest, DT.predict(xtest))
metrics.auc(fpr_bay, tpr_bay)
#输出分类报告，Y的真实值，Y的预测标签
print(classification_report(ytest, DT.predict(xtest)))

DecisionTreeClassifier()

0.7032561837334741

0.5492816468261678

              precision    recall  f1-score   support

           0       0.82      0.80      0.81    116832
           1       0.27      0.29      0.28     29075

    accuracy                           0.70    145907
   macro avg       0.55      0.55      0.55    145907
weighted avg       0.71      0.70      0.71    145907



In [17]:
DT = tree.DecisionTreeClassifier(criterion="gini",
                            max_depth=3,
                            min_samples_split=100,
                            min_samples_leaf=100,
                            random_state=12345)
DT.fit(xtrain,ytrain)
DT.score(xtest,ytest)
print(classification_report(ytest, DT.predict(xtest)))

DecisionTreeClassifier(max_depth=3, min_samples_leaf=100, min_samples_split=100,
                       random_state=12345)

0.8007292316338489

              precision    recall  f1-score   support

           0       0.80      1.00      0.89    116832
           1       0.00      0.00      0.00     29075

    accuracy                           0.80    145907
   macro avg       0.40      0.50      0.44    145907
weighted avg       0.64      0.80      0.71    145907



In [18]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(oob_score=True)
RF.fit(xtrain,ytrain)
RF.score(xtest,ytest)
print(classification_report(ytest, RF.predict(xtest)))

RandomForestClassifier(oob_score=True)

0.8029635315646267

              precision    recall  f1-score   support

           0       0.81      0.99      0.89    116832
           1       0.55      0.07      0.12     29075

    accuracy                           0.80    145907
   macro avg       0.68      0.53      0.50    145907
weighted avg       0.76      0.80      0.74    145907



In [19]:
from lightgbm import LGBMClassifier
LGBM = LGBMClassifier(num_leaves=1000,n_estimators=200)
LGBM.fit(xtrain,ytrain)
LGBM.score(xtest,ytest)
print(classification_report(ytest, LGBM.predict(xtest)))

LGBMClassifier(n_estimators=200, num_leaves=1000)

0.8019354794492383

              precision    recall  f1-score   support

           0       0.82      0.97      0.89    116832
           1       0.51      0.12      0.19     29075

    accuracy                           0.80    145907
   macro avg       0.66      0.54      0.54    145907
weighted avg       0.76      0.80      0.75    145907



In [7]:
from lightgbm import plot_importance
#model_lgb.feature_importances_
plot_importance(LGBM)

NameError: name 'LGBM' is not defined

In [20]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()
xgb.fit(xtrain,ytrain)
xgb.score(xtest,ytest)
print(classification_report(ytest, xgb.predict(xtest)))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

0.8036077775569369

              precision    recall  f1-score   support

           0       0.81      0.98      0.89    116832
           1       0.54      0.10      0.16     29075

    accuracy                           0.80    145907
   macro avg       0.68      0.54      0.53    145907
weighted avg       0.76      0.80      0.74    145907



In [21]:
from sklearn.neural_network import MLPClassifier   #导入多层感知器
mlp= MLPClassifier(hidden_layer_sizes=(50, 25), activation='relu',)
mlp.fit(xtrain,ytrain)
mlp.score(xtest,ytest)
print(classification_report(ytest, mlp.predict(xtest)))

MLPClassifier(hidden_layer_sizes=(50, 25))

0.8007292316338489

              precision    recall  f1-score   support

           0       0.80      1.00      0.89    116832
           1       0.00      0.00      0.00     29075

    accuracy                           0.80    145907
   macro avg       0.40      0.50      0.44    145907
weighted avg       0.64      0.80      0.71    145907



In [4]:
data = train.copy()
data.drop(['id','employmentTitle','ficoRangeLow','ficoRangeHigh','title','policyCode'],axis=1,inplace=True)

In [6]:
cat_columns = data.select_dtypes(include='O').columns
data[cat_columns]

,grade,subGrade,employmentLength,issueDate,earliesCreditLine
0,E,E2,2 years,2014/7/1,Aug-01
1,D,D2,5 years,2012/8/1,May-02
2,D,D3,8 years,2015/10/1,May-06
3,A,A4,10+ years,2015/8/1,May-99
4,C,C2,NaN,2016/3/1,Aug-77
...,...,...,...,...,...
799995,C,C4,7 years,2016/7/1,Aug-11
799996,A,A4,10+ years,2013/4/1,May-89
799997,C,C3,10+ years,2015/10/1,Jul-02
799998,A,A4,10+ years,2015/2/1,Jan-94


In [5]:
from autogluon.tabular import TabularDataset,TabularPredictor
train_data=TabularDataset(data)
TP=TabularPredictor(label='isDefault').fit(train_data)
print(classification_report(train_data['isDefault'], TP.predict(train_data)))

No path specified. Models will be saved in: "AutogluonModels\ag-20230502_144631\"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230502_144631\"
AutoGluon Version:  0.7.0
Python Version:     3.10.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
Train Data Rows:    800000
Train Data Columns: 40
Label Column: isDefault
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class

KeyboardInterrupt: 